# Test App Usage Pipeline

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from pathlib import Path
import sys, os

sys.path.append(os.path.join(os.getcwd(), "src"))

from pipeline import ImageProcessor
from detection import YOLODetector
from ocr import PaddleOCRWrapper, TextCorrector
from output import CSVGenerator

detector = YOLODetector()
ocr = PaddleOCRWrapper()
json_path = Path("data/scraped_app_names-cleaned.json") 
corrector = TextCorrector(json_path=json_path)

[2025/04/23 00:38:53] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/anh8878/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/anh8878/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_

In [2]:
DATA_DIR = Path("data")
IMAGES_DIR = DATA_DIR / "images"  
OUTPUT_DIR = Path("data")
DEBUG_DIR = OUTPUT_DIR / "debug_visualizations"

IMAGES_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DEBUG_DIR.mkdir(exist_ok=True)

processor = ImageProcessor(
    detector=detector,
    ocr=ocr,
    corrector=corrector,
    output_folder=str(OUTPUT_DIR)  
)

## 1. Single Image Processing

In [3]:
def process_and_display(image_path):
    image_path = Path(image_path)
    result = processor.process_image(str(image_path), debug=True)
    
    debug_img_path = DEBUG_DIR / f"debug_{image_path.name}"
    debug_img = cv2.imread(str(debug_img_path))
    
    if debug_img is not None:
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(debug_img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()
    else:
        print(f"Debug image not found at: {debug_img_path}")
    
    display(pd.DataFrame(result.extracted_data))
    return result

image_selector = widgets.Dropdown(
    options=sorted([str(p) for p in IMAGES_DIR.glob("*.[pj][np]g")]),
    description="Select Image:"
)

process_btn = widgets.Button(description="Process Image")
output = widgets.Output()

def on_process_click(b):
    with output:
        clear_output()
        process_and_display(image_selector.value)

process_btn.on_click(on_process_click)
display(widgets.VBox([image_selector, process_btn, output]))

## 2. Batch Processing

In [4]:
def run_batch_processing(input_dir, config_path="configs/settings.yaml"):
    from tqdm.notebook import tqdm
    import pandas as pd
    from utils.config import load_config
    
    config = load_config(Path(config_path))
    
    csv_config = {
        'required_columns': ['session_id', 'id', 'app_name', 'app_usage', 'total_usage'],
        'error_value': '-1',
        'time_format': 'compact'
    }
    
    all_results = []
    image_paths = list(Path(input_dir).glob("*.png")) + list(Path(input_dir).glob("*.jpg"))
    
    for img_path in tqdm(image_paths, desc="Processing"):
        try:
            result = processor.process_image(str(img_path))
            all_results.extend(result.extracted_data)
        except Exception as e:
            print(f"Error processing {img_path.name}: {str(e)}")
    
    # Initialize CSVGenerator
    csv_gen = CSVGenerator(csv_config)
    csv_path = Path(config.output.csv_path)
    csv_gen.generate({"batch_run": all_results}, str(csv_path))
    
    return pd.read_csv(csv_path)


batch_dir = widgets.Text(
    value=str(IMAGES_DIR),
    description="Input Directory:",
    disabled=False
)
run_batch_btn = widgets.Button(description="Run Batch")
batch_output = widgets.Output()

def on_batch_click(b):
    with batch_output:
        clear_output()
        df = run_batch_processing(batch_dir.value)
        display(df.head())

run_batch_btn.on_click(on_batch_click)
display(widgets.VBox([batch_dir, run_batch_btn, batch_output]))

## 3. Component-Level Testing

In [5]:
def test_ocr_correction(text, region_type):
    corrected, corrections = corrector.correct_text(text, region_type)
    print(f"Original: {text}")
    print(f"Corrected: {corrected}")
    print(f"Corrections Applied: {corrections}")

ocr_test_text = widgets.Text(
    value="1h30m",
    description="Test Text:"
)
ocr_test_type = widgets.Dropdown(
    options=["app_name", "app_usage"],
    value="app_usage",
    description="Region Type:"
)
ocr_test_btn = widgets.Button(description="Test OCR")
ocr_test_out = widgets.Output()

def on_ocr_test(b):
    with ocr_test_out:
        clear_output()
        test_ocr_correction(ocr_test_text.value, ocr_test_type.value)

ocr_test_btn.on_click(on_ocr_test)
display(widgets.VBox([ocr_test_text, ocr_test_type, ocr_test_btn, ocr_test_out]))

## 4. Visualization

In [6]:
def show_detections(image_path):
    img = cv2.imread(str(image_path))  
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return
    
    detections = detector.detect(img)
    
    for det in detections:
        cv2.rectangle(img, 
                     (det.x, det.y), 
                     (det.x + det.w, det.y + det.h), 
                     (0, 255, 0), 2)
        cv2.putText(img, 
                   f"{det.label} {det.confidence:.2f}", 
                   (det.x, det.y - 10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 
                   0.5, (0, 255, 0), 2)
    
    plt.figure(figsize=(10, 6))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(f"Detected {len(detections)} objects")
    plt.axis('off')
    plt.show()

viz_selector = widgets.Dropdown(
    options=sorted([str(p) for p in IMAGES_DIR.glob("*.[pj][np]g")]), 
    description="Select Image:"
)

viz_btn = widgets.Button(description="Show Detections")
viz_out = widgets.Output()

def on_viz_click(b):
    with viz_out:
        clear_output()
        show_detections(viz_selector.value)

viz_btn.on_click(on_viz_click)
display(widgets.VBox([viz_selector, viz_btn, viz_out]))